# Prevedere il crime rate

## Caricamento Librerie
Per prima cosa carichiamo le librerie per effettuare operazioni sui dati

*   NumPy per creare e operare su array a N dimensioni
*   pandas per caricare e manipolare dati tabulari
*   matplotlib per creare grafici

Importiamo le librerie usando i loro alias convenzionali

In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

## Caricamento dei dati
I dataset presi sono satti scricati dal sito ufficile delle nazioni unite (da completare)

## Crimini per furto

In [134]:
import os.path
file = "/content/data_cts_corruption_and_economic_crime.csv";
if not os.path.exists(file):
    print("Missing dataset");
else:
  theftCrime = pd.read_csv(file, index_col=False, encoding='latin1');

In [135]:
theftCrime.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22754 entries, 0 to 22753
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Iso3_code            22754 non-null  object 
 1   Country              22754 non-null  object 
 2   Region               22754 non-null  object 
 3   Subregion            22754 non-null  object 
 4   Indicator            22754 non-null  object 
 5   Dimension            22754 non-null  object 
 6   Category             22754 non-null  object 
 7   Sex                  22754 non-null  object 
 8   Age                  22754 non-null  object 
 9   Year                 22754 non-null  int64  
 10  Unit of measurement  22754 non-null  object 
 11  VALUE                22754 non-null  float64
 12  Source               22754 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 16.5 MB


In [136]:
theftCrime.head()

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
0,ARM,Armenia,Asia,Western Asia,Offences,by type of offence,Corruption,Total,Total,2013,Counts,782.0,CTS
1,AUT,Austria,Europe,Western Europe,Offences,by type of offence,Corruption,Total,Total,2013,Counts,3439.0,CTS
2,CHE,Switzerland,Europe,Western Europe,Offences,by type of offence,Corruption,Total,Total,2013,Counts,4884.0,CTS
3,CHL,Chile,Americas,Latin America and the Caribbean,Offences,by type of offence,Corruption,Total,Total,2013,Counts,339.0,CTS
4,COL,Colombia,Americas,Latin America and the Caribbean,Offences,by type of offence,Corruption,Total,Total,2013,Counts,23483.0,CTS


In [137]:
theftCrime.shape

(22754, 13)

In [138]:
theftCrime.describe()

,Year,VALUE
count,22754.000000,2.275400e+04
mean,2015.348159,2.675275e+04
std,5.059154,2.134465e+05
min,2003.000000,0.000000e+00
25%,2013.000000,6.000000e+00
50%,2017.000000,1.272963e+02
75%,2019.000000,1.484817e+03
max,2022.000000,7.026802e+06


## Preprocessing dei dati
Filtrimo il dataframe utilizzando come unità solo 'Counts' inevce che 'Rate per 100,000 population'. Effettuiamo quindi una prima scrematura dei dati

In [139]:
theftCrime = theftCrime[theftCrime['Unit of measurement'] != "Rate per 100,000 population"]

In [140]:
theftCrime.shape

(11377, 13)

Analizzando il dataframe, le feature 'Age', 'Sex', 'Iso3_code', 'Dimension', 'Unit of measurement'e 'Source' non sono rilevanti per il nostro modello, perciò effetuiamo la loro rimozione.

In [141]:
theftCrime = theftCrime.drop(['Unit of measurement', 'Sex', 'Iso3_code', 'Age', 'Source', 'Dimension', 'Indicator'], axis=1)

In [142]:
theftCrime.head()

,Country,Region,Subregion,Category,Year,VALUE
0,Armenia,Asia,Western Asia,Corruption,2013,782.0
1,Austria,Europe,Western Europe,Corruption,2013,3439.0
2,Switzerland,Europe,Western Europe,Corruption,2013,4884.0
3,Chile,Americas,Latin America and the Caribbean,Corruption,2013,339.0
4,Colombia,Americas,Latin America and the Caribbean,Corruption,2013,23483.0
